In [1]:
import requests
import json
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key

blockbusters_df = pd.read_csv("data/blockbusters.csv")
moviebudget_df= pd.read_csv("data/Movie_budget.csv")

movies_df = pd.merge(moviebudget_df,blockbusters_df, how = "right", on = "Title")
movies_df = movies_df.dropna(axis=0)
movies_df = movies_df.drop(["Worldwide_gross"], axis=1)
movies_df

,Rank,Release Date,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,length,rank_in_year,rating,Studio,Year
0,48,13-Feb-18,Black Panther,200000000,700059566,1336494320,Action,Adventure,Drama,7.4,135,1,PG-13,Walt Disney Pictures,2018
1,5,25-Apr-18,Avengers: Infinity War,300000000,678815482,2048359754,Action,Adventure,Sci-Fi,8.5,156,2,PG-13,Walt Disney Pictures,2018
2,50,15-Jun-18,Incredibles 2,200000000,608581744,1242805359,Animation,Action,Adventure,7.8,118,3,PG,Pixar,2018
3,144,6-Jun-18,Jurassic World: Fallen Kingdom,170000000,417719760,1308323302,Action,Adventure,Drama,6.2,129,4,PG-13,Universal Pictures,2018
7,11,23-May-18,Solo: A Star Wars Story,275000000,213767512,393151347,Action,Adventure,Drama,7.0,135,8,PG-13,Walt Disney Pictures,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,44,14-Dec-05,King Kong,207000000,218080025,550517357,Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976
451,"2,585",17-Dec-76,King Kong,"$23,000,000","$52,614,445","$90,614,445",Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976
452,"5,922",7-Apr-33,King Kong,"$672,000","$10,000,000","$10,001,781",Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976
455,445,7-Nov-19,Midway,100000000,56846802,126815030,History,Drama,Action,6.7,132,10,PG,Universal Pictures,1976


In [2]:
#Retrieving Data from OMDb 
#Adding director and actors
title= movies_df["Title"].tolist()
url = "http://www.omdbapi.com/?apikey=" + api_key + "&t="
response = []

for movie in title:
    movie_data = requests.get(url + movie).json()
    
    try:
        directors = movie_data["Director"]
    except:
        diretors = "N/A"


    response.append({ "Title":movie,
                    "Director": directors})
  
    print(f"{movie} was directed by {directors}")



Black Panther was directed by Ryan Coogler
Avengers: Infinity War was directed by Anthony Russo, Joe Russo
Incredibles 2 was directed by Brad Bird
Jurassic World: Fallen Kingdom was directed by J.A. Bayona
Solo: A Star Wars Story was directed by Ron Howard
Venom was directed by Ruben Fleischer
Venom was directed by Ruben Fleischer
Wonder Woman was directed by Patty Jenkins
Jumanji: Welcome to the Jungle was directed by Jake Kasdan
Spider-Man: Homecoming was directed by Jon Watts
It was directed by Andy Muschietti
Thor: Ragnarok was directed by Taika Waititi
Justice League was directed by Zack Snyder
Rogue One: A Star Wars Story was directed by Gareth Edwards
Captain America: Civil War was directed by Anthony Russo, Joe Russo
Deadpool was directed by Tim Miller
Zootopia was directed by Byron Howard, Rich Moore, Jared Bush
Batman v Superman: Dawn of Justice was directed by Zack Snyder
Sing was directed by Garth Jennings, Christophe Lourdelet
Sing was directed by Garth Jennings, Christoph

In [3]:
director_df = pd.DataFrame(response)
director_df.to_csv("data/director.csv", index_label="Movie ID")
director_df

,Title,Dirtector
0,Black Panther,Ryan Coogler
1,Avengers: Infinity War,"Anthony Russo, Joe Russo"
2,Incredibles 2,Brad Bird
3,Jurassic World: Fallen Kingdom,J.A. Bayona
4,Solo: A Star Wars Story,Ron Howard
...,...,...
237,King Kong,Peter Jackson
238,King Kong,Peter Jackson
239,King Kong,Peter Jackson
240,Midway,Roland Emmerich


In [4]:
allmovies_df = pd.merge(movies_df,director_df, how = "left", on = "Title")
allmovies_df.to_csv("data/allmoviesdata.csv", index_label="Movie ID")
allmovies_df

,Rank,Release Date,Title,Production Budget,Domestic Gross,Worldwide Gross,Main Genre,2nd Genre,3rd Genre,IMDB Rating,length,rank_in_year,rating,Studio,Year,Dirtector
0,48,13-Feb-18,Black Panther,200000000,700059566,1336494320,Action,Adventure,Drama,7.4,135,1,PG-13,Walt Disney Pictures,2018,Ryan Coogler
1,5,25-Apr-18,Avengers: Infinity War,300000000,678815482,2048359754,Action,Adventure,Sci-Fi,8.5,156,2,PG-13,Walt Disney Pictures,2018,"Anthony Russo, Joe Russo"
2,50,15-Jun-18,Incredibles 2,200000000,608581744,1242805359,Animation,Action,Adventure,7.8,118,3,PG,Pixar,2018,Brad Bird
3,144,6-Jun-18,Jurassic World: Fallen Kingdom,170000000,417719760,1308323302,Action,Adventure,Drama,6.2,129,4,PG-13,Universal Pictures,2018,J.A. Bayona
4,11,23-May-18,Solo: A Star Wars Story,275000000,213767512,393151347,Action,Adventure,Drama,7.0,135,8,PG-13,Walt Disney Pictures,2018,Ron Howard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,"5,922",7-Apr-33,King Kong,"$672,000","$10,000,000","$10,001,781",Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976,Peter Jackson
302,"5,922",7-Apr-33,King Kong,"$672,000","$10,000,000","$10,001,781",Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976,Peter Jackson
303,"5,922",7-Apr-33,King Kong,"$672,000","$10,000,000","$10,001,781",Horror,Fantasy,Adventure,5.8,134,7,PG,Paramount Pictures,1976,Peter Jackson
304,445,7-Nov-19,Midway,100000000,56846802,126815030,History,Drama,Action,6.7,132,10,PG,Universal Pictures,1976,Roland Emmerich


Changes in Cost Over the Years

* main - line

* 1990's

* 2000's

* 2010's

Production Costs per Genre
* pie chart - broken by genre
* pie chart - production companies

Production Cost vs Domestic and Worldwide Gross
* production cost vs domestic - scatter plot
* production cost vs worldwide

Production Cost vs Viewer Ratings
* scatter plot

Director
* bar - combined gross for movies

* top 100 movies directors